In [1]:
import numpy as np
import pythonToC
import parsing_yacc 
from parsing_lex import MyLexer
import pythonTopython2
import time

In [2]:
# Define test input
input_code = """
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

c = np.add(a, b)
d = np.subtract(a, b)
e = np.sum(a)
f = np.dot(a, b)
"""

In [3]:
# Create an instance of MyLexer
lexer = MyLexer()

# Build the lexer
lexer.build()

# Test the lexer on the input code
lexer.test(input_code)

LexToken(VARIABLE,'a',1,1)
LexToken(EQUALS,'=',1,3)
LexToken(ARRAY,'np.array',1,5)
LexToken(LPAREN,'(',1,13)
LexToken(LIST,'[1, 2, 3]',1,14)
LexToken(RPAREN,')',1,23)
LexToken(VARIABLE,'b',1,25)
LexToken(EQUALS,'=',1,27)
LexToken(ARRAY,'np.array',1,29)
LexToken(LPAREN,'(',1,37)
LexToken(LIST,'[4, 5, 6]',1,38)
LexToken(RPAREN,')',1,47)
LexToken(VARIABLE,'c',1,50)
LexToken(EQUALS,'=',1,52)
LexToken(NP_ADD,'np.add',1,54)
LexToken(LPAREN,'(',1,60)
LexToken(VARIABLE,'a',1,61)
LexToken(COMMA,',',1,62)
LexToken(VARIABLE,'b',1,64)
LexToken(RPAREN,')',1,65)
LexToken(VARIABLE,'d',1,67)
LexToken(EQUALS,'=',1,69)
LexToken(NP_SUBTRACT,'np.subtract',1,71)
LexToken(LPAREN,'(',1,82)
LexToken(VARIABLE,'a',1,83)
LexToken(COMMA,',',1,84)
LexToken(VARIABLE,'b',1,86)
LexToken(RPAREN,')',1,87)
LexToken(VARIABLE,'e',1,89)
LexToken(EQUALS,'=',1,91)
LexToken(NP_SUM,'np.sum',1,93)
LexToken(LPAREN,'(',1,99)
LexToken(VARIABLE,'a',1,100)
LexToken(RPAREN,')',1,101)
LexToken(VARIABLE,'f',1,103)
LexToken(EQUALS,'=',1

In [4]:
def deploy_parser(input, mode): 
    output = ""

    for line in input.split("\n"):
        if line == "":
            continue
        # print("line: ", line)   
        result = parsing_yacc.main(line, mode)
        # print("result: ", result)
        # Check if the result is a string 
        if isinstance(result, str):
            output += result + "\n"

    return output

In [5]:
# Generate C code from Python input using yacc
print("Generating C code...")
#measure time to genreate C code - 
start_time = time.time()

c_output = deploy_parser(input_code, "C")
print("C code generation complete.")

c_time  = time.time() - start_time
print("Time taken to generate C code:", c_time)
print("\n")

print("c_output:", c_output)

if c_output:
    # Set up files for conversion
    pythonToC.set_up("test")
    pythonTopython2.setup("test")

    # Execute the input code to define variables
    exec(input_code)

    # Generate C code from Python input using pythontoC functions
    for line in input_code.split('\n'):
        if '=' in line:
            variable, value = map(str.strip, line.split("="))
            if '[' in value:  # Check if value is an array
                array_length = len(eval(value))  # Evaluate the array and get its length
                pythonToC.set_arr(variable, value, array_length, "test")
            else:  # Single value
                pythonToC.set_arr(variable, value, 1, "test")



    # Generate Python code from Python input using pythontopython functions
    for line in input_code.split('\n'):
        if '=' in line:
            exec("pythonTopython2." + line.strip())
    p_time  = time.time() - start_time
    

    # Generate Python code from C input using pythontopython functions
    #for line in c_output.split('\n'):
        #if line.strip():
            #exec("pythonTopython2." + line.strip())

    print("Conversion complete.")
else:
    print("No C output generated.")


Generating C code...
C code generation complete.
Time taken to generate C code: 0.0050601959228515625


c_output:    float h_a[1, 2, 3] = [1, 2, 3];
    float *d_a;
   cudaMalloc(&d_a,1, 2, 3*sizeof(float);
    cudaMemcpy(d_a, h_a,sizeof(h_a), cudaMemcpyHostToDevice);


   float h_b[4, 5, 6] = [4, 5, 6];
    float *d_b;
   cudaMalloc(&d_b,4, 5, 6*sizeof(float);
    cudaMemcpy(d_b, h_b,sizeof(h_b), cudaMemcpyHostToDevice);


addArraysHelper(a, b, c);
subtractArraysHelper(a, b, d);
sumArraysHelper(a, e);

Time taken to generate Python code: 7.605552673339844e-05
Conversion complete.
